In [6]:
setwd("/home/pa/Documents/github/doc_suomi/code")
source("utils.R")
source("data_cook.R")
cat(howto_data)

`summarise()` has grouped output by 'album_id', 'name'. You can override using the `.groups` argument.



How to use datasets (Personal library)
 
call    base()             for real values
call    z_scored()         for normalized
call    min_maxed()        for normalized2
call    upsampled_album()  for binded upsamplped albums
call    low_minmax()       for min-maxed low level
call    low_z()            for normalized low level
call    low_raw()          for raw  low level

# Writting data for low

In [7]:
oi = low_minmax()

# Removing albums with unusually small tracks_lengths
oi %<>% 
    group_by(track_id) %>%
    mutate(nsamples = NROW(valence)) %>%
    filter(nsamples < 5)

a_out <- unique(oi$album_id)
data = low_minmax()

data %<>%
    filter(!album_id %in% a_out) %>% 
    ungroup() %>%
    group_by(track_id) %>%
    mutate(track_section = seq(1, NROW(track_id), 1))

In [9]:
# Performing quantile cuts
data %<>%
    filter(!album_id %in% a_out) %>% 
    ungroup() %>%
    group_by(track_id) %>%
    mutate(track_section = seq(1, NROW(track_id), 1)) %>%
    mutate(track_section = lsr::quantileCut(track_section, 5, labels = c("a", "b", "c", "d", "e"))) %>%
    ungroup() %>% group_by(album_id, track_id, track_section) %>%
    #summarizing by quantile section
    summarise(danceability = mean(danceability),
              energy = mean(energy), 
              loudness_overall = mean(loudness_overall),
              speechiness = mean(speechiness),
              acousticness = mean(acousticness),
              instrumentalness = mean(instrumentalness), 
              liveness = mean(liveness),
              valence = mean(valence),
              tempo_overall = mean(tempo_overall), 
              duration_ms = mean(duration_ms),
              time_signature = mean(time_signature),
              track_number = mean(track_number), 
              loudness_continuous = mean(loudness_continuous),
              tempo_continuous = mean(tempo_continuous), 
              tempo_confidence = mean(tempo_confidence), 
              key_confidence = mean(key_confidence),
              mode_confidence = mean(mode_confidence), 
              time_signature_confidence = mean(time_signature_confidence)) %>%
    ungroup() %>%
    arrange(album_id) %>% 
    select(!track_section)

`summarise()` has grouped output by 'album_id', 'track_id'. You can override using the `.groups` argument.



In [10]:
# Calculating cumulative track features
data %>%
    group_by(album_id, track_id) %>%
    summarise(danceability_cum = mean(danceability),
           energy_cum = mean(energy),
           loudness_overall_cum = mean(loudness_overall),
           speechiness_cum = mean(speechiness),
           acousticness_cum = mean(acousticness),
           instrumentalness_cum = mean(instrumentalness), 
           liveness_cum = mean(liveness),
           valence_cum = mean(valence),
           tempo_overall_cum = mean(tempo_overall), 
           duration_ms_cum = mean(duration_ms),
           time_signature_cum = mean(time_signature),
           loudness_continuous_cum = mean(loudness_continuous),
           tempo_continuous_cum = mean(tempo_continuous),
           tempo_confidence_cum = mean(tempo_confidence),
           key_confidence_cum = mean(key_confidence),
           mode_confidence_cum = mean(mode_confidence),
           time_signature_confidence_cum = mean(time_signature_confidence)) %>%

    ungroup() %>% group_by(album_id) %>%
    mutate(danceability_cum = cumsum(danceability_cum),
           energy_cum = cumsum(energy_cum),
           loudness_overall_cum = cumsum(loudness_overall_cum),
           speechiness_cum = cumsum(speechiness_cum),
           acousticness_cum = cumsum(acousticness_cum),
           instrumentalness_cum = cumsum(instrumentalness_cum), 
           liveness_cum = cumsum(liveness_cum),
           valence_cum = cumsum(valence_cum),
           tempo_overall_cum = cumsum(tempo_overall_cum), 
           duration_ms_cum = cumsum(duration_ms_cum),
           time_signature_cum = cumsum(time_signature_cum),
           loudness_continuous_cum = cumsum(loudness_continuous_cum),
           tempo_continuous_cum = cumsum(tempo_continuous_cum),
           tempo_confidence_cum = cumsum(tempo_confidence_cum),
           key_confidence_cum = cumsum(key_confidence_cum),
           mode_confidence_cum = cumsum(mode_confidence_cum),
           time_signature_confidence_cum = cumsum(time_signature_confidence_cum)) -> cummed

oi = merge(x = cummed, y = data, by.x = c("album_id", "track_id"), by.y = c("album_id", "track_id"))

`summarise()` has grouped output by 'album_id'. You can override using the `.groups` argument.



In [11]:
#Min-maxing new variables
oi %<>% 
    ungroup() %>%
    mutate(danceability_cum = minmax(danceability_cum, 1, 2),
           energy_cum = minmax(energy_cum, 1, 2),
           loudness_overall_cum = minmax(loudness_overall_cum, 1, 2),
           speechiness_cum = minmax(speechiness_cum, 1, 2),
           acousticness_cum = minmax(acousticness_cum, 1, 2),
           instrumentalness_cum = minmax(instrumentalness_cum, 1, 2), 
           liveness_cum = minmax(liveness_cum, 1, 2),
           valence_cum = minmax(valence_cum, 1, 2),
           tempo_overall_cum = minmax(tempo_overall_cum, 1, 2), 
           duration_ms_cum = minmax(duration_ms_cum, 1, 2),
           time_signature_cum = minmax(time_signature_cum, 1, 2),
           loudness_continuous_cum = minmax(loudness_continuous_cum, 1, 2),
           tempo_continuous_cum = minmax(tempo_continuous_cum, 1, 2),
           tempo_confidence_cum = minmax(tempo_confidence_cum, 1, 2),
           key_confidence_cum = minmax(key_confidence_cum, 1, 2),
           mode_confidence_cum = minmax(mode_confidence_cum, 1, 2),
           time_signature_confidence_cum = minmax(time_signature_confidence_cum, 1, 2))

In [12]:
# Tagging each track as greater-smaller than the previous track.
oi %>% group_by(album_id) %>%
    mutate(valence_cat  = greater(valence),
           energy_cat   = greater(energy),
           loudness_cat = greater(loudness_overall),
           tempo_cat    = greater(tempo_overall)) %>%
    ungroup() %>% 
    group_by(album_id, track_number) %>%
    mutate(valence_cat = valence_cat[1],
           energy_cat = energy_cat[1],
           loudness_cat = loudness_cat[1],
           tempo_cat = tempo_cat[1]) %>%
    write.csv("/home/pa/Documents/github/doc_suomi/data/lstm/lstm.csv")

# Writing data for Markov model

In [6]:
dt = base() %>% 
        group_by(album_id) %>%
        mutate(valence = greater(valence),
               energy = greater(energy),
               loudness = greater(loudness),
               tempo = greater(tempo))

In [7]:
dt %>%
    write.csv("/home/pa/Documents/github/doc_suomi/data/markov/markov.csv")

# Writting data for optimization

Same as markov but already discreticized

In [ ]:
dt = base() %>% 
        group_by(album_id) %>%
        mutate(valence = greater(valence),
               energy = greater(energy),
               loudness = greater(loudness),
               tempo = greater(tempo))

write.csv(dt, "/home/pa/Documents/github/doc_suomi/data/optimization/data_discrete.csv")